# Imputation with R/qtl2

Made by: Sarah Odell

github: https://github.com/sarahodell

Summary: This is a workflow for setting up, running and analyzing the accuracy of R/qtl2 for calculating genotype probabilities of simulated maize MAGIC double haploids using parental genotype data. R/qtl2 documentation can be found <a href='https://kbroman.org/qtl2/docs.html'>here</a>. This is done only for chromosome 10. It is assumed that the founder lines and the lines to be imputed are entirely homozygous. The imputation accuracy is quantified as the percentage of imputed blocks assigned to the correct parental haplotype. 

Table of Contents:
1. <a href='#section_1'>Setting up control files</a>
2. <a href='#section_2'>Running qtl2</a>
3. <a href='#section_3'>Analyzing genotype probabilities</a>
4. <a href='#section_4'>Assessing imputation accuracy</a>


The simulated MAGIC lines used here were generated in [BiogemmaSimulation](../BiogemmaSimulation.ipynb)


In [1]:
print(format(Sys.time(), "Last updated: %m/%d/%Y"))
print(sprintf("Created using: %s",R.Version()$version.string))

[1] "Last updated: 07/10/2018"
[1] "Created using: R version 3.4.2 (2017-09-28)"


### Installing qtl2

Information on installing qtl2 can be found in the <a href='https://kbroman.org/qtl2/assets/vignettes/user_guide.html'>User Guide</a>

In [ ]:
library("qtl2")
library("tidyverse")
library("dplyr")
library("data.table")
library("ggplot2")
library("reshape2")
library("tibble")

In [22]:
# Set wkdir to whatever your working directory is
wkdir='~/Documents/PBGG/MAGIC/qtl2'
setwd(wkdir)

<a href='section_1'></a>

### 1. Setting up control files

R/qtl2 requires input files to be set up in a very particular way (See User Guide for more info). The files needed for getting genotype probabilities are:
- A genotype file of the individuals you want to impute
- A founder genotype file of the parents
- A cross information file on how the lines were developed
- A genetic map for the markers in the genotype file
- A physical map for the markers in the genotype file
- A json control file linking all these files together

To start, we'll need vcf files with the genotype information of the founders and the lines to be imputed, as well as a genetic map (Ogut, 2015).

For this, we will need bcftools installed (Version 1.2 was used here).


In [11]:
#The python script genofile2.py creates the genotype data from the vcf and converts it into a csv
print(system('python genofile2.py -h',intern=T))

 [1] "usage: genofile2.py [-h] infile outfile"                                    
 [2] ""                                                                           
 [3] "Program description: Takes a vcf file and converts it to a csv format with" 
 [4] "markers as columns and samples as rows with nucleotide information in IUPAC"
 [5] "format. This csv file is formatted for use with R/qtl2"                     
 [6] ""                                                                           
 [7] "positional arguments:"                                                      
 [8] "  infile      The input vcf file"                                           
 [9] "  outfile     The output csv filename"                                      
[10] ""                                                                           
[11] "optional arguments:"                                                        
[12] "  -h, --help  show this help message and exit"                              


In [ ]:
genos='MAGICSim_062018/MAGICSim_BiogemmaFull_062018.vcf.gz'
genofile='MAGICSim_062018/chr10/MAGICSim_geno.csv'
system(sprintf("python genofile2.py %s %s",genos,genofile)) #Create the genotype file

For the founder lines, the vcf file is converted into a temporary csv, which is then used to create another, properly formatted csv files for the founder genotype file and the physical map. The header variable should be a string like 'id,chr,pos,sample1,sample2, ...'

In [12]:
foundergenos='biogemma/BiogemmaFounders_600K_Genotypes.vcf.gz'
fgenofile='MAGICSim_062018/chr10/MAGICSim_foundergeno_c10.csv'
tmp='bigemma/founders_tmp.csv'
header="id,chr,pos,A632_usa,B73_inra,CO255_inra,FV252_inra,OH43_inra,A654_inra,FV2_inra,C103_inra,EP1_inra,D105_inra,W117_inra,B96,DK63,F492,ND245,VA85"

In [ ]:
system(sprintf('echo $%s > %s',header,tmp))
system(sprintf("bcftools query -f '%%ID,%%CHROM,%%PIS[,%%GT]\n' %s >> %s",foundergenos,tmp))

In [21]:
print(system(sprintf('python foundergeno.py -h'),intern=T))

 [1] "usage: foundergeno.py [-h] infile outfile"                                  
 [2] ""                                                                           
 [3] "Program description: Takes a vcf file and converts it to a csv format with" 
 [4] "founder rows and marker genotypes as columns. Genotypes are assumed"        
 [5] "homozygous, with reference allele as A and alternate alleles as B. This csv"
 [6] "file is formatted for use with R/qtl2"                                      
 [7] ""                                                                           
 [8] "positional arguments:"                                                      
 [9] "  infile      The input vcf file"                                           
[10] "  outfile     The output csv filename"                                      
[11] ""                                                                           
[12] "optional arguments:"                                                        
[13]

In [ ]:
system(sprintf('python foundergeno.py %s %s',tmp,fgenofile)

Now we should have properly formated genotype files for both the founder lines and the simulated MAGIC lines. To get the physical map for the genetic markers in the genotype files, simply pull some columns from the tmp file.

In [33]:
pmapfile='MAGICSim_062018/chr10/MAGICSim_pmap_c10.csv'
print(system(sprintf('python pmap.py -h'),intern=T))

 [1] "usage: pmap.py [-h] infile outfile"                                        
 [2] ""                                                                          
 [3] "Program description: Generates a physical map file in csv format. This csv"
 [4] "file is formatted for use with R/qtl2"                                     
 [5] ""                                                                          
 [6] "positional arguments:"                                                     
 [7] "  infile      The input intermediate csv file generated by bcftools query" 
 [8] "  outfile     The output csv filename"                                     
 [9] ""                                                                          
[10] "optional arguments:"                                                       
[11] "  -h, --help  show this help message and exit"                             


In [ ]:
system(sprintf('python pmap.py %s %s',tmp,pmapfile )

In [27]:
pmap=fread('MAGICSim_062018/chr10/MAGICSim_pmap_c10.csv',data.table=F)
head(pmap)

marker,chr,pos
AX-91807321,10,0.275461
AX-91157290,10,0.276497
AX-91807318,10,0.276688
AX-91157288,10,0.279348
AX-91157305,10,0.301249
AX-90617680,10,0.301380


The generate the genetic map from the physical map, we will load the genetic map from Ogut,2015, and extrapolate the genetic positions of the markers from this.

ogutmap<-fread('ogut_map.csv',data.table=F)
ogutmap$SNP_ID = as.character(ogutmap$SNP_ID)
str(ogutmap)

In [36]:
#Create approxfun 
get_cM<-approxfun(x=ogutmap$pos,y=ogutmap$cM,method="linear",yleft=-7,yright=107) 
#yleft and yright were set as arbitrary boundaries for markers that extend beyond the range of the
#ogut map

#apply to pmap
pmap$cM = get_cM(pmap$pos)
gmap<-pmap[,c('marker','chr','cM')]
names(gmap)<-c('marker','chr','pos')
head(gmap)

marker,chr,pos
AX-91807321,10,-7
AX-91157290,10,-7
AX-91807318,10,-7
AX-91157288,10,-7
AX-91157305,10,-7
AX-90617680,10,-7


In [ ]:
#Write out to file
gmapfile="MAGICSim_062018/chr10/MAGICSim_gmap_c10.csv"
write.csv(gmap10,file=gmapfile,row.names = FALSE,quote = FALSE)

The cross info file is structured with the founder lines as columns and individuals as rows. 
The values are numbers for the order of the crosses. For example:

A cross order of (A x B) x (C x D) for 3 lines would be structured as


ind,A,B,C,D

i1,1,2,3,4

i2,1,2,3,4

i3,1,2,3,4

Finally, the json control file needs to be set up. See the User Guide for more help with this.
Below is what mine ended up looking like:

In [42]:
print(system('cat MAGICSim_062018/chr10/MAGICSim.json',intern=T))

 [1] "{"                                                                                                                                                                                                                       
 [2] "    \"description\":  \"10 Simulated MAGIC lines from 16 Biogemma founders\","                                                                                                                                           
 [3] "    \"crosstype\": \"riself16\","                                                                                                                                                                                        
 [4] "    \"sep\": \",\","                                                                                                                                                                                                     
 [5] "    \"na.strings\": \"NA\","                                                                      

<a href='section_2'></a>

### 2. Running qtl2

Now that we have the files set up, we can run qtl2. The calculation of genotype probabilities 
is computationally intensive. The use of multiple cores is recommended.

In [43]:
magic<-read_cross2('MAGICSim_062018/chr10/MAGICSim.json')

In [ ]:
pr<-calc_genoprob(magic,error_prob=0.002,cores=0) #calculate genotype probabilities
apr<-genoprob_to_alleleprob(pr,cores=0) #get allele probabilities

In [ ]:
saveRDS(pr,"geno_probs.rds")
saveRDS(apr,"allele_probs.rds")

<a href='section_3'></a>

### 3. Analyzing genotype probabilities

Now that we have genotype probabilities, it's time to visualize them and compare them to the known probabilities.


In [45]:
pr = readRDS('MAGICSim_062018/geno_probs.rds')
apr = readRDS('MAGICSim_062018/allele_probs.rds')
pmap10 = read.csv('MAGICSim_062018/chr10/MAGICSim_pmap_c10.csv')

In [46]:
hex_colors=c("#f42896","#84ef7c","#a8bc44","#8ed1d6","#702349","#f2875b","#28ad26","#afd3ef","#937266",
"#56cc59","#663dd3","#478959","#47145b","#7c2126","#ad147a","#afb735")
founders=c("A632_usa","B73_inra","CO255_inra","FV252_inra","OH43_inra","A654_inra",
           "FV2_inra","C103_inra","EP1_inra","D105_inra","W117_inra","B96","DK63","F492","ND245","VA85")


In [47]:
pr_c <- clean_genoprob(pr,value_threshold = 0.0001) #Set very low values to zero (1e-4)

In [60]:
all_pr=c()
for (m in 1:10){
  md <- pr_c[[1]][m,,]
  tm <- t(md)
  mdf <- as.data.frame(tm,row.names = rownames(tm))
  mdf<-rownames_to_column(mdf,"marker")
  names(mdf)=c("marker",founders)
  mdf <- merge(mdf,pmap10,by.x='marker',by.y='marker')
  mdf <- mdf[,c(2:17,19)]
  mlong<-melt(mdf,id="pos")
  mlong$sample=rep(sprintf("M%s",m),dim(mlong)[1])
  all_pr=rbind(all_pr,mlong)
}

In [ ]:
ggplot(data=all_pr,aes(x=pos,y=value,color=variable)) + facet_grid(sample ~ .) +scale_color_manual(values=hex_colors) + scale_fill_manual(values=hex_colors)+ geom_area(aes(fill=variable),alpha=5/10) + geom_line() + ggtitle("Genotype Probability for Sample M1") + xlab("Position (Mb)") + ylab("Probability") + guides(color=FALSE)

![image](../data_files/Biogemma_qtl2_genoprob.png)

In [62]:
options(warn=-1)
all_breaks=c()
for (m in 1:10){
  md <- pr_c[[1]][m,,] #pull out the probs for one individual
  tm <- t(md) #transpose matrix
  mdf <- as.data.frame(tm,row.names = rownames(tm))
  mdf<-rownames_to_column(mdf,"marker")
  names(mdf)=c("marker",founders)
  mdf <- merge(mdf,pmap10,by.x='marker',by.y='marker') #get the physical marker positions
  mdf <- mdf[,c(2:17,19)]
  mlong<-melt(mdf,id="pos") #melt the dataframe
  
  max_prob<-mlong %>% group_by(pos) %>% top_n(1,value) #grab the highest probability parent for each position
  max_prob<-as.data.frame(max_prob)
  max_prob<-max_prob[order(max_prob$pos),] #sort in ascending order
  rownames(max_prob)=seq(1:nrow(max_prob))
  
  break_start=max_prob[1,'pos']*1e6 #convert from Mb to bp
  break_donor=as.character(max_prob[1,'variable'])
  break_end=c()
  #Find locations where the most probable donor changes
  last_pos=max_prob[1,'pos']
  last_donor=as.character(max_prob[1,'variable'])
  for(row in 1:nrow(max_prob)){
    current_start=max_prob[row,'pos']
    current_donor=as.character(max_prob[row,'variable'])
    if(current_donor != last_donor){
      break_start=rbind(break_start,current_start*1e6)
      break_donor=rbind(break_donor,current_donor)
      break_end=rbind(break_end,(last_pos*1e6))
    }
    last_pos=max_prob[row,'pos']
    last_donor=as.character(max_prob[row,'variable'])
  }
  break_end=rbind(break_end,max(max_prob$pos)*1e6)
  line=rep(sprintf('M%s',m),dim(break_donor)[1])
  chr=rep(10,dim(break_donor)[1])
  #convert into dataframe and append to all_breaks
  breaks<-data.frame('sample'=line,'chr'=chr,'start'=break_start,'end'=break_end,'donor1'=break_donor,'donor2'=break_donor)
  all_breaks=rbind(all_breaks,breaks)
}
options(warn=0)

In [54]:
head(all_breaks)

sample,chr,start,end,donor1,donor2
M1,10,275461,17969501,VA85,VA85
M1,10,18007502,133905897,DK63,DK63
M1,10,133905966,145003452,F492,F492
M1,10,145004071,146974993,D105_inra,D105_inra
M1,10,147005255,149602124,B73_inra,B73_inra
M2,10,275461,10889066,W117_inra,W117_inra


In [ ]:
write.table(all_breaks,file='MAGICSim_062018/Predicted_BiogemmaMAGIC_qtl2.txt',sep='\t',row.names = FALSE,quote = FALSE)

In [69]:
colors=cbind(founders,hex_colors)
write.table(colors,file='Founder_colorcodes.txt',col.names=FALSE,row.names=FALSE,quote=FALSE,sep='\t')

In [70]:
system("python ../impute/pscripts/ideogram.py --colors Founder_colorcodes.txt  --title 'qtl2 Predicted Genotypes' Predicted_BiogemmaMAGIC_qtl2.txt qtl2_image.png")

And here is a plot of the genotype probabilities predicted by qtl2

Actual | Predicted
- | - 
![alt](../data_files/ActualBiogemmaMAGICImage062018.png) | ![alt](../data_files/qtl2_image.png)

In [74]:
print(system('python ../impute/pscripts/intersect.py ../impute/sim_files/062018/Biogemma_MAGICSim.txt Predicted_BiogemmaMAGIC_qtl2.txt',intern=T))

  [1] "None"                                                                          
  [2] "None"                                                                          
  [3] "None"                                                                          
  [4] "None"                                                                          
  [5] "None"                                                                          
  [6] "Parent W117_inra not shared between actual and predicted in sample M1 chr 10"  
  [7] ""                                                                              
  [8] "Parent A632_usa not shared between actual and predicted in sample M1 chr 10"   
  [9] ""                                                                              
 [10] "Parent ND245 not shared between actual and predicted in sample M1 chr 10"      
 [11] ""                                                                              
 [12] "Parent B96 not shared between actual

In [80]:
print(system('cat intersect_output.txt | tail -4',intern=T))

[1] ""                                             
[2] "Total Percentage Correct: 0.998"              
[3] "Percentage of Homozygous Calls Correct: 0.998"
[4] "Percentage of Heterozygous Calls: 0.0"        
